In [25]:
import re
import pandas as pd
from collections import Counter
import scores

In [7]:
movie_ann_df = pd.read_csv("/workspace/coref/data/annotation/movies.txt", index_col=None, header=None, sep="\s", engine="python")
movie_ann_df.columns = ["movie", "annotator"]
print(movie_ann_df)

                movie  annotator
0    avengers_endgame    prithvi
1  dead_poets_society     chakor
2           john_wick    prithvi
3            prestige     chakor
4         quiet_place  athashree
5            zootopia  athashree


In [8]:
annotation_df = pd.read_csv("/workspace/coref/data/annotation/csv/avengers_endgame.csv", index_col=None)

In [9]:
annotation_df

,begin,end,begin_segment,end_segment,begin_line,end_line,leftContext,surface,rightContext,entityNum,entityLabel,entityGroup,DIFFICULT,SPEAKER,DIFFICULT.1,APPOSITION,NOMINAL,PRONOUN
0,7830,7836,NaN,NaN,-1,-1,restrictions set forth above.\n,ROCKET,F O R Y O U R C O N S I D,0,ROCKET,False,False,True,False,False,False,False
1,7884,7887,NaN,NaN,-1,-1,restrictions set forth above.\n,his,F O R Y O U R C O N S I D,0,ROCKET,False,False,True,False,False,False,True
2,9487,9493,NaN,NaN,-1,-1,restrictions set forth above.\n,ROCKET,F O R Y O U R C O N S I D,0,ROCKET,False,False,True,False,False,False,False
3,9543,9549,NaN,NaN,-1,-1,restrictions set forth above.\n,Rocket,F O R Y O U R C O N S I D,0,ROCKET,False,False,True,False,False,False,False
4,9591,9597,NaN,NaN,-1,-1,restrictions set forth above.\n,ROCKET,F O R Y O U R C O N S I D,0,ROCKET,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5020,200329,200333,NaN,NaN,-1,-1,restrictions set forth above.\n,WONG,F O R Y O U R C O N S I D,69,WONG,False,False,True,False,False,False,False
5021,96030,96038,NaN,NaN,-1,-1,restrictions set forth above.\n,A1 CLINT,F O R Y O U R C O N S I D,70,CLINT-2012,False,False,True,False,False,False,False
5022,97567,97575,NaN,NaN,-1,-1,restrictions set forth above.\n,A1 CLINT,F O R Y O U R C O N S I D,70,CLINT-2012,False,False,True,False,False,False,False
5023,97582,97589,NaN,NaN,-1,-1,restrictions set forth above.\n,himself,F O R Y O U R C O N S I D,70,CLINT-2012,False,False,True,False,False,False,True


In [13]:
sorted(Counter(annotation_df["entityLabel"]).items())

[('A FEMALE SHIELD\nAGENT', 10),
 ('AGENT SITWELL', 17),
 ('AKIHIKO', 22),
 ('ALEXANDER\nPIERCE', 21),
 ('ANCIENT ONE', 66),
 ('BAD NEBULA', 131),
 ('BROCK RUMLOW', 12),
 ('BRUCE BANNER', 372),
 ('BUCKY', 14),
 ('CAROL DANVERS', 95),
 ('CASSIE LANG', 13),
 ('CLINT BARTON', 284),
 ('CLINT-2012', 4),
 ('DR. STEPHEN STRANGE', 33),
 ('DRAX', 7),
 ('EBONY MAW', 21),
 ('EDWIN JARVIS', 8),
 ('FRIGGA', 70),
 ('Friday', 13),
 ('GAMORA', 78),
 ('GRIEVING MAN', 16),
 ('GROOT', 17),
 ('HANK PYM', 30),
 ('HAPPY HOGAN', 8),
 ('HOPE VAN DYNE', 14),
 ('HOWARD STARK', 67),
 ('HULK-2012', 14),
 ('JANE FOSTER', 20),
 ('KORG', 14),
 ('LAURA BARTON', 19),
 ('LILA BARTON', 25),
 ('LOKI', 8),
 ('LOKI-2012', 33),
 ('M.P', 6),
 ('MANTIS', 7),
 ('MARIA HILL', 1),
 ('MIEK', 3),
 ('MORGAN STARK', 63),
 ('NATASHA ROMANOV', 223),
 ('NATASHA-2012', 6),
 ('NEBULA', 171),
 ('NICK FURY', 4),
 ('OKOYE', 10),
 ('OLD STEVE', 35),
 ('Odin', 2),
 ('PEGGY\nCARTER', 30),
 ('PEPPER POTTS', 66),
 ('PETER PARKER', 62),
 ('PETER 

In [14]:
annotation_df.columns

Index(['begin', 'end', 'begin_segment', 'end_segment', 'begin_line',
       'end_line', 'leftContext', 'surface', 'rightContext', 'entityNum',
       'entityLabel', 'entityGroup', 'DIFFICULT', 'SPEAKER', 'DIFFICULT.1',
       'APPOSITION', 'NOMINAL', 'PRONOUN'],
      dtype='object')

In [22]:
for _, row in movie_ann_df.iterrows():
    movie, annotator = row["movie"], row["annotator"]
    if movie == "avengers_endgame":
        screenplay = open("/workspace/coref/data/annotation/screenplay/avengers_endgame_cleaned.txt").read()
    else:
        screenplay = open("/workspace/coref/data/annotation/screenplay/{}.txt".format(movie)).read()
    annotation_df = pd.read_csv("/workspace/coref/data/annotation/csv/{}.csv".format(movie), index_col=None, dtype={"begin": int, "end": int})
    
    print(movie)

    for character, character_df in annotation_df.groupby("entityLabel"):
        character = re.sub("\s+", " ", character).strip()
        print("\t", character)
        for _, row in character_df.sample(n=min(10, len(character_df))).iterrows():
            begin, end = row["begin"], row["end"]
            mention = screenplay[begin: end]
            mention = re.sub("\s+", " ", mention).strip()
            print("\t\t", mention)
    print()

avengers_endgame
	 A FEMALE SHIELD AGENT
		 FEMALE SHIELD AGENT
		 FEMALE SHIELD AGENT
		 THE FEMALE AGENT
		 You
		 She
		 FEMALE SHIELD AGENT
		 The Agent
		 I
		 A FEMALE SHIELD AGENT
		 FEMALE SHIELD AGENT
	 AGENT SITWELL
		 AGENT SITWELL
		 your
		 I
		 AGENT SITWELL
		 AGENT SITWELL
		 AGENT SITWELL
		 Sitwell
		 his
		 Sitwell
		 Sitwell
	 AKIHIKO
		 AKIHIKO
		 You
		 Akihiko’s
		 Akihiko
		 HIM
		 AKIHIKO
		 AKIHIKO
		 you
		 A YAKUZA BOSS, AKIHIKO
		 me
	 ALEXANDER PIERCE
		 me
		 ALEXANDER PIERCE
		 Alexander Pierce
		 ALEXANDER PIERCE
		 ALEXANDER PIERCE
		 My
		 Pierce
		 ALEXANDER PIERCE
		 He
		 ALEXANDER PIERCE
	 ANCIENT ONE
		 I
		 ANCIENT ONE
		 THE ANCIENT ONE
		 I
		 HER
		 her
		 ANCIENT ONE
		 She
		 THE ANCIENT ONE
		 you
	 BAD NEBULA
		 BAD NEBULA
		 (BAD) NEBULA
		 HER
		 her
		 Bad Nebula
		 NEBULA
		 Daughter
		 BAD NEBULA
		 SHE
		 you
	 BROCK RUMLOW
		 Rumlow
		 BROCK RUMLOW
		 RUMLOW’S
		 Rumlow
		 Rumlow
		 Rumlow
		 Rumlow
		 Rumlow
		 BROCK RUMLOW
		 Rum

In [26]:
athashree = pd.read_csv("/workspace/coref/data/annotation/validation/athashree.csv")
chakor = pd.read_csv("/workspace/coref/data/annotation/validation/chakor.csv")
prithvi = pd.read_csv("/workspace/coref/data/annotation/validation/prithvi.csv")
reference = pd.read_csv("/workspace/coref/data/annotation/validation/reference.csv")

athashree_clusters = []
chakor_clusters = []
prithvi_clusters = []
reference_clusters = []

for _, character_df in athashree.groupby("entityLabel"):
    cluster = set()
    for _, row in character_df.iterrows():
        begin, end = int(row["begin"]), int(row["end"])
        cluster.add((begin, end))
    athashree_clusters.append(cluster)

for _, character_df in chakor.groupby("entityLabel"):
    cluster = set()
    for _, row in character_df.iterrows():
        begin, end = int(row["begin"]), int(row["end"])
        cluster.add((begin, end))
    chakor_clusters.append(cluster)

for _, character_df in prithvi.groupby("entityLabel"):
    cluster = set()
    for _, row in character_df.iterrows():
        begin, end = int(row["begin"]), int(row["end"])
        cluster.add((begin, end))
    prithvi_clusters.append(cluster)

for _, character_df in reference.groupby("entityLabel"):
    cluster = set()
    for _, row in character_df.iterrows():
        begin, end = int(row["begin"]), int(row["end"])
        cluster.add((begin, end))
    reference_clusters.append(cluster)

In [29]:
print(scores.conll2012(reference_clusters, athashree_clusters))
print(scores.conll2012(reference_clusters, chakor_clusters))
print(scores.conll2012(reference_clusters, prithvi_clusters))

0.8219707532241335
0.8387907269684429
0.8642722510377332
